# Run

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
print("tensorflow version:",tf.__version__)

import wandb
from wandb.tensorflow import WandbHook

tensorflow version: 2.2.0-dlenv


## wandb config and initialization

In [2]:
wandb.init(name='sparky',
           group='0',
           project='whether-predictor',
           sync_tensorboard=True,
          )

W&B Run: https://app.wandb.ai/jonjonchu/whether-predictor/runs/fmrqweud

In [3]:
config = {"epochs": 4,
          "frames": 1,
          "channels": 1,
          "pixels_x": 21,
          "pixels_y": 21,
          "input_shape": (1, 21, 21),
          "filters": [8, 16, 16],
          "kernel_sizes": [(5,5), (3,3), (3,3)],
          "strides": [(1,1), (1,1), (1,1),],
          "bias_init": 0.1,
          "weight_decay": 1e-5,
         }
wandb.config.update(config)

## Create architecture

### Input processing

In [5]:
input_ = layers.Input(name="model_input",
                      shape = wandb.config.input_shape)

In [6]:
input_noised = layers.GaussianNoise(0.1, name="input_noised")(input_)

### Convolutional Stack

In [7]:
name = "conv_"
num_layers = 3

In [8]:
for i in range(num_layers):
    # Create layers
    conv_cell = layers.Conv2D(name="conv_{}".format(i+1),
                             data_format='channels_first',
                             filters=wandb.config.filters[i],
                             kernel_size=wandb.config.kernel_sizes[i],
                             strides=wandb.config.strides[i],
                             kernel_initializer=tf.keras.initializers.GlorotNormal(),
                             activity_regularizer=tf.keras.regularizers.l2(l=wandb.config.weight_decay),
                             activation="relu",
                             )
    bn_cell = layers.BatchNormalization(axis=1, name="conv_bn_{}".format(i+1))
    
    # Link layers together into stack
    # Connect previous layer (either input_noised or stack) to conv_cell
    if i == 0:
        stack =  conv_cell(input_noised)
    else:
        stack = conv_cell(stack)
    # Connect conv_cell to bn unless at end of loop
    if i != num_layers-1:
        stack = bn_cell(stack)
    elif i == num_layers-1:
        conv_output = bn_cell(stack)

### Encoder

In [9]:
encoder_input = tf.expand_dims(conv_output, 0)

In [10]:
encoder_cell = layers.ConvLSTM2D(name="encoder{}".format(i+1),
                                 filters = wandb.config.filters[-1],
                                 kernel_size=wandb.config.kernel_sizes[0],
                                 padding='same',
                                 data_format='channels_first',
                                 return_sequences=True,
                                 return_state=True)
_, state_h, state_c = encoder_cell(encoder_input)

encoder_states = [state_h, state_c]

**Save convolutional stack output shape in configs.**

In [11]:
tuple(conv_output.shape)

(None, 16, 13, 13)

In [12]:
wandb.config.update({"conv_output_shape": tuple(conv_output.shape)})

### Decoder

In [13]:
decoder_input = tf.expand_dims(conv_output, 0, name="decoder_input")

In [14]:
decoder_cell = layers.ConvLSTM2D(name="decoder{}".format(i+1),
                                   filters=wandb.config.filters[-1],
                                   kernel_size=wandb.config.kernel_sizes[0],
                                   padding='same',
                                   data_format='channels_first',
                                   return_sequences=True,
                                   return_state=True)

decoder_output, _, _ = decoder_cell(decoder_input, initial_state = encoder_states)



In [15]:
reshaped_decoder_output = tf.reshape(decoder_output,
                                     shape=[-1,
                                            wandb.config.filters[-1],
                                            decoder_output.shape[-2],
                                            decoder_output.shape[-1]],
                                     name="decoder_reshaped",
                                     )

### Deconvolutional Stack

In [31]:
rev_filters = wandb.config.filters[::-1]
rev_filters = rev_filters[1:] + [wandb.config.channels]
rev_kernel_sizes = wandb.config.kernel_sizes[::-1]
rev_strides = wandb.config.strides[::-1]
print(rev_filters)
print(wandb.config.filters)
print(rev_kernel_sizes)
print(wandb.config.kernel_sizes)
print(rev_strides)
print(wandb.config.strides)

[16, 8, 1]
[8, 16, 16]
[[3, 3], [3, 3], [5, 5]]
[[5, 5], [3, 3], [3, 3]]
[[1, 1], [1, 1], [1, 1]]
[[1, 1], [1, 1], [1, 1]]


In [35]:
for i in range(num_layers):
    # Create layers
    deconv_cell = layers.Conv2DTranspose(name="deconv_{}".format(i+1),
                             data_format='channels_first',
                             filters=rev_filters[i],
                             kernel_size=rev_kernel_sizes[i],
                             strides=rev_strides[i],
                             kernel_initializer=tf.keras.initializers.GlorotNormal(),
                             activity_regularizer=tf.keras.regularizers.l2(l=wandb.config.weight_decay),
                             activation="relu",
                             )
    bn_cell = layers.BatchNormalization(axis=1, name="deconv_bn_{}".format(i+1))
    
    # Link layers together into stack
    # Connect previous layer (either input_noised or stack) to conv_cell
    if i == 0:
        stack =  deconv_cell(reshaped_decoder_output)
    else:
        stack = deconv_cell(stack)
    # Connect conv_cell to bn unless at end of loop
    if i != num_layers-1:
        stack = bn_cell(stack)
    else:
        deconv_output = bn_cell(stack)

In [36]:
output = tf.expand_dims(deconv_output, 0)

In [37]:
full_model = tf.keras.Model(name="Full_stack",
                       inputs = input_,
                       outputs = output)
full_model.summary()

Model: "Full_stack"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
model_input (InputLayer)        [(None, 1, 21, 21)]  0                                            
__________________________________________________________________________________________________
input_noised (GaussianNoise)    (None, 1, 21, 21)    0           model_input[0][0]                
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 8, 17, 17)    208         input_noised[0][0]               
__________________________________________________________________________________________________
conv_bn_1 (BatchNormalization)  (None, 8, 17, 17)    32          conv_1[0][0]                     
_________________________________________________________________________________________